# Focused Coding
___

## Table of Content

1. [Libraries](#libraries)
2. [Data Preprocessing](#preprocessing-data-and-grouping)
3. [Focused Coding Helper](#focused-coding----sample-helper)
___

## Libraries

All libraries which are needed to execute the code are listed here. Install the packages by using the `requirements.txt` file. 

The documentation can be found in the [README.md](README.md) file.

In [3]:
# import packages
import pandas as pd 
import os
import numpy as np
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import nltk
from preprocessing_functions import *

## Preprocessing Data and Grouping

In [4]:
# load data
df = pd.read_csv('data/comments_final.csv')
df.head(3)

,video_id,published_at,like_count,text,author
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,"The answer is if China and India don't help, i...",0.0
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,"and that guy is an expert, we're screwed",1.0
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,Kennedy is a gem.,2.0


In [5]:
# group data by author and see distribution of comments 
df['author'] = pd.to_numeric(df['author'], errors='coerce').astype('Int64')
summary = df.groupby('author').agg(
    count=('author', 'size'),
    unique_video_id_count=('video_id', 'nunique')
).reset_index()
summary.sort_values(by='count', ascending=False, inplace=True)

# print top 20 authors
latex_table = summary.head(20).to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
author & count & unique_video_id_count \\
\midrule
17605 & 206 & 206 \\
3743 & 188 & 124 \\
18119 & 166 & 161 \\
17604 & 155 & 138 \\
18380 & 154 & 154 \\
17977 & 142 & 138 \\
17906 & 139 & 115 \\
17676 & 135 & 103 \\
17755 & 134 & 134 \\
14017 & 129 & 126 \\
14057 & 128 & 123 \\
2610 & 125 & 125 \\
25732 & 116 & 88 \\
1645 & 107 & 62 \\
17581 & 106 & 104 \\
6165 & 103 & 52 \\
6 & 99 & 98 \\
1106 & 97 & 75 \\
17608 & 92 & 80 \\
17590 & 87 & 79 \\
\bottomrule
\end{tabular}



In [6]:
# extract original text for seeing in comparising capitalized words etc.
extracted_col = df["text"]

# process data with using functions from functions.py
processed_df = (
    df.pipe(remove_users, 'text')
      .pipe(lowercase_text, 'text')
      .pipe(remove_whitespace, 'text')
      .pipe(remove_punctuation, 'text')
)

In [7]:
# Add the extracted column to the second DataFrame
processed_df = pd.concat([processed_df, extracted_col.rename("og_text")], axis=1)
processed_df.head(3)

,video_id,published_at,like_count,text,author,og_text
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,the answer is if china and india dont help it ...,0,"The answer is if China and India don't help, i..."
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,and that guy is an expert were screwed,1,"and that guy is an expert, we're screwed"
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,kennedy is a gem,2,Kennedy is a gem.


In [8]:
# use lemmatization to reduce words to their root form
processed_df['text'] = processed_df['text'].astype('str')
processed_df = lemmatize_words(processed_df, 'text')

In [9]:
processed_df.lemmatized_text = processed_df.lemmatized_text.apply(lambda x: '' if str(x) == 'nan' else x)

## Focused Coding -- Sample helper

this helper is written to find comments with important keywords we could firstly find through exploring the comments or through topic modeling and Word2Vec.

How does it work?

- Input: Insert a lemmatized keyword of interest or multiple in `substrings` or `string`
- Ouput: Get comments as output which are talking about those keywords to get a variety of comments based on that topic or word of interest.

In [12]:
substrings = ['god plan', 'greenhouse gas', 'natural cycle', 'hoax']
string = ['miniscule']
pattern = '|'.join(string)

In [13]:
filtered_df = processed_df[processed_df['lemmatized_text'].str.contains(pattern, case=False, na=False)]
pd.set_option('display.max_colwidth', None)
print(len(filtered_df))
filtered_df.sample(n=10)

17


video_id          published_at  like_count  \
56756  xkw3NYo2RJc  2023-09-18T18:12:12Z           2   
59097  o7o59q8qMak  2023-07-05T11:07:46Z           0   
57729  cTlcXyUVHjw  2023-08-11T09:04:05Z           7   
33141  pZJZAuli46I  2024-02-19T09:17:47Z           0   
61289  xd6I8YCnHrQ  2023-07-06T14:47:36Z           1   
84951  vEFmVgjdLfs  2023-03-09T04:27:49Z           0   
81401  AGaNghtj41I  2023-12-27T14:38:54Z           1   
30933  T_APkKd8qt0  2023-09-23T09:11:19Z           0   
13289  uMahtFMivwo  2024-01-23T17:17:52Z           0   
85972  zadtR8-dFHw  2024-03-18T02:40:30Z           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 